# Check your background of numerical programming

This notebook is a quick recap of the main practical concepts of numerical programming, and introduces the Casadi module of the Pinocchio library:
- Pinocchio is obviously a library gathering the main rigid-body dynamics library.
- Casadi is an auto-differentiation tools, i.e. you can write symbolic expression and differentiate them. It is also wrapping the IpOpt solver, which is a quite robust and generic constraint-based optimization solver. You can then quickly write a problem with some symbolic expression for the cost and constraints, and Casadi will transfer it to IpOpt for solving it.
- The Pinocchio/Casadi module is a part of the Pinocchio solver written by Justin Carpentier which uses Casadi to unroll the algorithms of Pinocchio into symbolic expression, which then can be differentiated. It is a very powerful tool for initiation and prototyping, although it cannot scales to very large problems (in particular whole-body optimization problems) due to various technical problems that we can discuss.

## Definition of the toy problem studied here

We propose to explain how to define and solve inverse-geometry problems, i.e. problems where the main decision variable is a single robot configuration.

Inverse geometry is the problem of computing a configuration $q$ of a robot model that satisfies a set of 'tasks', which we formalize as error functions $e(q)$ that are zero when the task is achieved.

- $q \in \mathcal{C}$: configuration of the robot.
- $e_i(q) \in \mathbb{R}$: error function for task $i$. In this tutorial our error function is real valued.

Typical tasks for limbed robots include putting a foot/end-effector on a surface, moving the center of mass (CoM) to a target location, etc. For example, an error function for a foot position task could be:

$$
e(q) = \|p_\mathrm{foot}^* - p_\mathrm{foot}(q)\|
$$

In this tutorial, we will define a task function $e(q)$ for the end-effector pose of a manipulator and solve the inverse geometry problem

$$
\underset{q}{\mathrm{minimize}}\ e(q)
$$

using an off-the-shelf nonlinear optimizer that evaluates gradients by finite differences (slower), then a more efficient formulation using CasADi (faster).

## Before we start...

%load is a "magic" command of Jupyter. When we will invite you to think to a solution by yourselves, we will indicate it by the %do_not_load command. Feel free to transform it to a plain %load if you want the answer. This is define in the following import.

In [1]:
import gepetuto.magic

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide
    the solutions to some questions. Change it for %load if you want to see (and
    execute) the solution.


Let's import everything we will need in this tutorial:

In [2]:
# %load tp1/generated/invgeom_imports
import time
import unittest
import example_robot_data as robex
import numpy as np
import casadi
import pinocchio as pin
import pinocchio.casadi as cpin
from wan2024.meshcat_viewer_wrapper import MeshcatVisualizer, colors


## Set up

Let us start by loading the UR5 robot model from `example-robot-data` and define its neutral configuration as follows:

In [3]:
# %load tp1/generated/invgeom_robot
robot = robex.load("ur10")
model = robot.model
data = robot.data


We will also use the MeshCat visualizer to watch the optimization unfold. Let's set up a visualizer for our robot model:

In [4]:
# %load tp1/generated/invgeom_visualizer
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)


*** You asked to start meshcat "classically" in tcp://127.0.0.1:6000
*** Did you start meshcat manually (meshcat-server)
Wrapper tries to connect to server <tcp://127.0.0.1:6000>
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [5]:
viz.display(robot.q0*3)

In [6]:
viz.viewer.jupyter_cell()

## Definition of the problem

### Task target

Our task will be to move the end effector *frame* of the UR5 to a desired pose (pose := position + orientation). For this purpose, we define:

- The name of the corresponding frame, which is ``"tool0"`` for the UR5.
- The target pose of this frame, which we define by an affine transform ${}_{world} M_{target} \in SE(3)$ (also known as a "placement").

The robot will start from an arbitrary initial configuration (hint: to make the solver's life easier, avoid the singular $[0\ 0\ \ldots\ 0]$ configuration) $q_0$.

In [7]:
# %load tp1/generated/invgeom_task_params
robot.q0 = np.array([0, -np.pi / 2, 0, 0, 0, 0])

tool_id = model.getFrameId("tool0")

in_world_M_target = pin.SE3(
    pin.utils.rotate("x", np.pi / 4),
    np.array([-0.5, 0.1, 0.2]),
)


### Sidetrack: visualization



In [8]:
# %load tp1/generated/invgeom_visualizer_callback
# --- Add box to represent target
# Add a vizualization for the target
boxID = "world/box"
viz.addBox(boxID, [0.05, 0.1, 0.2], [1.0, 0.2, 0.2, 0.5])
# Add a vizualisation for the tip of the arm.
tipID = "world/blue"
viz.addBox(tipID, [0.08] * 3, [0.2, 0.2, 1.0, 0.5])

def displayScene(q):
    """
    Given the robot configuration, display:
    - the robot
    - a box representing tool_id
    - a box representing in_world_M_target
    """
    pin.framesForwardKinematics(model, data, q)
    in_world_M_tool = data.oMf[tool_id]
    viz.applyConfiguration(boxID, in_world_M_target)
    viz.applyConfiguration(tipID, in_world_M_tool)
    viz.display(q)
    time.sleep(1e-1)


We are now ready to optimize for real!

## Introducting Casadi

We wil now define our problem in [CasADi](https://web.casadi.org), a framework that allows us to formulate problems as computation graphs from which it can compute derivatives (notably first-order derivatives, *a.k.a.* gradients, and second-order derivatives, *a.k.a.* Hessians) automatically. This feature is called *automatic differentiation*.

### Modelling with CasADi

Pinocchio, in the version 3 we are using in today's tutorial, provides a `pinocchio.casadi` interface to represent configurations (such as $q \in \mathcal{C}$ or $M \in SE(3)$) in CasADi computation graphs, and differentiate them appropriately.

CasADi being designed for Euclidean variables, differentiating properly over manifolds is not as straightforward as representing everything as matrices and vectors in $\mathbb{R}^n$ and taking their usual derivatives. Pinocchio allows us to take derivatives over manifolds properly, without worrying about what happens below the surface. To be able to do that, we first need to redefine our model as a CasADi one:

In [9]:
# %load tp1/generated/invgeom_casadi_model
# --- Casadi helpers
cmodel = cpin.Model(model)
cdata = cmodel.createData()


Our configuration variable $q$ is now a CasADi ``SX`` symbol of size $n_q$:

In [10]:
# %load tp1/generated/invgeom_cq
cq = casadi.SX.sym("q", model.nq, 1)


In [11]:
3*(cq+1)

SX(@1=3, @2=1, [(@1*(q_0+@2)), (@1*(q_1+@2)), (@1*(q_2+@2)), (@1*(q_3+@2)), (@1*(q_4+@2)), (@1*(q_5+@2))])

We complete the computation graph from ``cmodel`` to ``cq`` by calling the corresponding forward-kinematics function:

In [12]:
# %load tp1/generated/invgeom_casadi_fk
cpin.framesForwardKinematics(cmodel, cdata, cq)


In [13]:
cdata.oMf[tool_id]

SE3(SX(@1=cos(q_0), @2=4.89664e-12, @3=cos(q_1), @4=sin(q_1), @5=((@2*@3)-@4), @6=(@1*@5), @7=cos(q_2), @8=((@2*@4)+@3), @9=(@1*@8), @10=sin(q_2), @11=((@6*@7)-(@9*@10)), @12=cos(q_3), @13=sin(q_3), @14=((@2*@12)-@13), @15=((@6*@10)+(@9*@7)), @16=(@12+(@2*@13)), @17=((@11*@14)-(@15*@16)), @18=cos(q_4), @19=sin(q_0), @20=sin(q_4), @21=((@17*@18)-(@19*@20)), @22=cos(q_5), @23=((@2*@13)+@12), @24=((@2*@12)-@13), @25=((@11*@23)+(@15*@24)), @26=sin(q_5), @27=(@19*@5), @28=(@19*@8), @29=((@27*@7)-(@28*@10)), @30=((@27*@10)+(@28*@7)), @31=((@29*@14)-(@30*@16)), @32=((@31*@18)+(@1*@20)), @33=((@29*@23)+(@30*@24)), @34=(@3+(@2*@4)), @35=((@2*@3)-@4), @36=((@34*@7)+(@35*@10)), @37=((@35*@7)-(@34*@10)), @38=((@36*@14)+(@37*@16)), @39=(@38*@18), @40=((@37*@24)-(@36*@23)), @41=0, @42=((@17*@20)+(@19*@18)), @43=((@21*@26)+(@25*@22)), @44=((@1*@18)-(@31*@20)), @45=((@32*@26)+(@33*@22)), @46=(@38*@20), @47=((@40*@22)-(@39*@26)), @48=0.612, @49=-0.1719, @50=0.220941, @51=0.5723, @52=0.1149, @53=0.1157,

### Task error

For a given robot configuration $q$, the end effector will be located at ${}_{world} M_{tool}(q)$, which is a function of $q$ we can compute with Pinocchio's forward kinematics function. The goal of our task will be to make ${}_{world} M_{tool}(q)$ coincide with ${}_{world} M_{target}$, which we can define mathematically as: minimize the (Lie-group) difference ${}_{world} M_{target} \ominus {}_{world} M_{tool}(q)$, or equivalently:

$$
\underset{q}{\mathrm{minimize}} \ e(q) := \| {}_{world} M_{target} \ominus {}_{world} M_{tool}(q) \|
$$

Remember the definition of the $\ominus$ (left) Lie-group difference? No? Let's write it in Python anyway! 

As a CasADi function, it writes:

In [14]:
# %load tp1/generated/invgeom_casadi_error
error_tool = casadi.Function(
    "etool",
    [cq],
    [
        cpin.log6(
            cdata.oMf[tool_id].inverse() * cpin.SE3(in_world_M_target)
        ).vector
    ],
)


In [15]:
cpin.log6(cpin.SE3(in_world_M_target)).vector

SX(@1=0, [-0.5, 0.173346, 0.150342, 0.785398, @1, @1])

Finally, we define a computation graph from our optimization variable to our error function:

In [16]:
# %load tp1/generated/invgeom_casadi_computation_graph
opti = casadi.Opti()
var_q = opti.variable(model.nq)
totalcost = casadi.sumsqr(error_tool(var_q))


In [17]:
var_q

MX(opti0_x_1)

In [18]:
error_tool(var_q)

MX(etool(opti0_x_1){0})

In [19]:
totalcost

MX(@1=etool(opti0_x_1){0}, dot(@1, @1))

### IPOPT optimization
We now specify a cost to minimize, and an algorithm to search for the solution.

In [20]:
# %load tp1/generated/invgeom_ipopt
opti.minimize(totalcost)
opti.solver("ipopt")  # select the backend solver
opti.callback(lambda i: displayScene(opti.debug.value(var_q)))
# %load tp1/generated/invgeom_solve
# Caution: in case the solver does not converge, we are picking the candidate values
# at the last iteration in opti.debug, and they are NO guarantee of what they mean.
try:
    sol = opti.solve_limited()
    sol_q = opti.value(var_q)
except:
    print("ERROR in convergence, plotting debug info.")
    sol_q = opti.debug.value(var_q)


In [21]:
#opti.subject_to(var_q >= 0)
#opti.subject_to(var_q[1] == 0)

Finally, we can call the solver:

In [22]:
# %load tp1/generated/invgeom_solve
# Caution: in case the solver does not converge, we are picking the candidate values
# at the last iteration in opti.debug, and they are NO guarantee of what they mean.
try:
    sol = opti.solve_limited()
    sol_q = opti.value(var_q)
except:
    print("ERROR in convergence, plotting debug info.")
    sol_q = opti.debug.value(var_q)



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.13, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality c

We can check that the solver converged to the same solution:

In [31]:
# %load tp1/generated/invgeom_check_final_placement
pin.framesForwardKinematics(model,data,sol_q)
print(
    "The robot finally reached effector placement at\n",
    data.oMf[tool_id]
)
robot.placement(sol_q,6),data.oMi[6]

The robot finally reached effector placement at
   R =
           1  7.77156e-16 -3.33067e-16
-6.66134e-16     0.707107    -0.707107
-3.88578e-16     0.707107     0.707107
  p = -0.5  0.1  0.2



(  R =
            1 -3.33067e-16 -7.77156e-16
 -6.66134e-16    -0.707107    -0.707107
 -3.88578e-16     0.707107    -0.707107
   p =     -0.5 0.165195 0.134805,
   R =
            1 -3.33067e-16 -7.77156e-16
 -6.66134e-16    -0.707107    -0.707107
 -3.88578e-16     0.707107    -0.707107
   p =     -0.5 0.165195 0.134805)

In [28]:
in_world_M_target

  R =
        1         0         0
        0  0.707107 -0.707107
        0  0.707107  0.707107
  p = -0.5  0.1  0.2

In [32]:
opti.value(error_tool(var_q)),pin.log(data.oMf[tool_id].inverse()*in_world_M_target).vector

(array([ 3.44169138e-15,  3.05311332e-15, -2.28983499e-15, -3.33066907e-16,
         3.33066907e-16,  7.45793918e-16]),
 array([ 3.44169138e-15,  3.05311332e-15, -2.28983499e-15, -3.33066907e-16,
         3.33066907e-16,  7.45793918e-16]))

In [26]:
sol_q

array([-3.77728452,  1.25943505, -2.44804804, -0.91798303,  0.96552974,
        2.37748027])

In [27]:
pin.SE3.__repr__=pin.SE3.__str__

# Probleme plus complexe

In [57]:
del var_q,opti,var_tau,var_f,cq,ctau,cf

NameError: name 'opti' is not defined

In [108]:
opti2 = casadi.Opti()
var_q = opti2.variable(model.nq)
var_tau = opti2.variable(model.nq)
var_f = opti2.variable(6)

In [109]:
cq = casadi.SX.sym("cq2",cmodel.nq,1)
ctau = casadi.SX.sym("ctau",cmodel.nq,1)
cf = casadi.SX.sym("cf",6,1)

In [110]:
cpin.framesForwardKinematics(cmodel,cdata,cq)
cpin.computeGeneralizedGravity(cmodel,cdata,cq)
cpin.computeJointJacobians(cmodel,cdata);

In [111]:
Jtool = cpin.getFrameJacobian(cmodel,cdata,tool_id,pin.LOCAL_WORLD_ALIGNED)

In [112]:
dyncst = casadi.Function(
    "dyn",
    [cq,ctau,cf],
    [cdata.g - ctau - Jtool.T@cf]
)

In [113]:
error3 = casadi.Function("e3",[cq],
                         [ cdata.oMf[tool_id].translation-
                          in_world_M_target.translation ])

In [114]:
fdes = np.array([ 10,0,0, 0,0,0])
totalcost = casadi.sumsqr(var_f-fdes) + 1e-3*casadi.sumsqr(var_tau)

In [115]:
opti2.subject_to( error3(var_q) == 0)
opti2.subject_to( dyncst(var_q,var_tau,var_f) == 0)

In [116]:
opti2.minimize(totalcost)
opti2.solver("ipopt")  # select the backend solver
opti2.callback(lambda i: displayScene(opti2.debug.value(var_q)))
# %load tp1/generated/invgeom_solve
# Caution: in case the solver does not converge, we are picking the candidate values
# at the last iteration in opti.debug, and they are NO guarantee of what they mean.
try:
    sol = opti2.solve_limited()
    sol_q = opti2.value(var_q)
    sol_tau = opti2.value(var_tau)
    sol_f = opti2.value(var_f)
except:
    print("ERROR in convergence, plotting debug info.")
    sol_q = opti2.debug.value(var_q)


This is Ipopt version 3.14.13, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:       84
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       61

Total number of variables............................:       18
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        9
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0000000e+02 1.21e+02 1.94e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [88]:
sol_tau

array([-5.01657669e-14, -3.37612891e-15, -4.65346857e-15,  1.74134256e-13,
        3.86103435e-14, -1.10845609e-14])

In [89]:
sol_f

array([ 1.00000000e+01,  4.66634504e-19, -1.85092845e-13, -1.96103774e-16,
        4.85834388e-13, -6.43526863e-17])

In [103]:
opti2.value(dyncst(var_q,0*var_tau,0*var_f))

array([-2.33863269e-18, -1.44026113e-12, -7.52749008e-13,  8.19785795e-14,
       -0.00000000e+00, -0.00000000e+00])

In [102]:
opti2.value(var_q)

array([ 1.41371669e+01, -4.71238898e+00, -1.66256038e-14, -7.85398163e+00,
       -2.82743339e+01,  2.56252579e+01])

In [104]:
grav = casadi.Function("grav", [cq],[ cdata.g ])

In [105]:
opti2.value(grav(var_q))

array([-2.33863269e-18, -1.44026113e-12, -7.52749008e-13,  8.19785795e-14,
        0.00000000e+00,  0.00000000e+00])

In [106]:
displayScene(sol_q)

In [107]:
pin.computeGeneralizedGravity(model,data,sol_q)

array([-2.33863269e-18, -1.44026113e-12, -7.52749008e-13,  8.19785795e-14,
        0.00000000e+00,  0.00000000e+00])